In [14]:
import torch
from torch import nn
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
import cProfile

In [15]:
x = torch.rand(1, 256, 16, 16, 16)
conv = nn.Conv3d(256, 128, 3)

In [16]:
inp, mat1, mat2 = torch.rand(64, 64), torch.rand(64, 64), torch.rand(64, 64)
addmm = torch.addmm
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    y = addmm(inp, mat1, mat2)
print(addmm)
print(prof.key_averages().table())


linear = nn.Linear(1024, 512)
linear.weight = torch.nn.Parameter(torch.rand_like(linear.weight))
x = torch.randn(1, 1024)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    y = linear(x)
    y.mean().backward()
print(linear)
print(prof.key_averages().table())

conv = nn.Conv3d(256, 128, 3)
conv.weight = torch.nn.Parameter(torch.rand_like(conv.weight))
x = torch.rand(1, 256, 16, 16, 16)
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    y = conv(x)
    y.mean().backward()
print(conv)
print(prof.key_averages().table())

<built-in method addmm of type object at 0x7f3325c69a40>
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
           aten::addmm        87.25%     130.000us       100.00%     149.000us     149.000us             1  
           aten::copy_        12.08%      18.000us        12.08%      18.000us      18.000us             1  
    aten::resolve_conj         0.67%       1.000us         0.67%       1.000us       0.333us             3  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 149.000us

Linear(in_features=1024, out_features=512, bias=True)
-------------------------------------------------------  ------------  ------------  ---------

[W CUDAFunctions.cpp:108] Warning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (function operator())
STAGE:2023-10-31 15:24:50 220917:220917 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-10-31 15:24:50 220917:220917 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-10-31 15:24:50 220917:220917 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2023-10-31 15:24:50 220917:220917 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-10-31 15:24:50 220917:220917 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-10-31 15:24:50 220917:220917 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2023-10-31 15:24:50 220917:220917 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2

In [4]:
print(prof.key_averages().table())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.28%     210.000us        72.24%      53.294ms      53.294ms             1  
                                      aten::convolution         0.05%      35.000us        71.96%      53.084ms      53.084ms             1  
                                     aten::_convolution         0.11%      80.000us        71.91%      53.049ms      53.049ms             1  
                               aten::mkldnn_convolution        71.75%      52.932ms        71.80%      52.969ms      52.969ms             1  
      